## Import Packages

In [46]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
import tensorflow as tf
import ntpath


## Import CSVs

In [47]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed'
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = ntpath.basename(file)
    dfs.append(df)

C:\Users\Dell\Documents\B.Sc IT (Hons.) Computing and Business\FYP - Luke Bezzina\Code\mlpLearning\..\..\Datasets\processed


# Splitting dataset in dependent and independent variables

In [48]:
def split_dataset(df):
    x = np.array(df.iloc[:, 1:-1].values, dtype=np.float)
    y = np.array(df.iloc[:, -1:].values)
    return x, y

# Encoding Dependent Variable

In [49]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y = np.array(le.fit_transform(y))

def encode_var(y):
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
    y = np.array(ct.fit_transform(y))
    print(y)

    return y

# Splitting dataset (training + testing)

In [50]:
def train_test_sp(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

    # print("Train - Independent Variables")
    # print(x_train)
    # print("Test - Independent Variables")
    # print(x_test)
    # print("Train - Dependent Variables")
    # print(y_train)
    # print("Test - Dependent Variables")
    # print(y_test)

    return x_train, x_test, y_train, y_test

# Building ANN with MLP

In [51]:
def machine_learning(x_train, y_train):
    ann = tf.keras.models.Sequential()
    # hidden layer
    ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
    ann.add(tf.keras.layers.Dense(units=3, activation='relu'))
    # output layer
    ann.add(tf.keras.layers.Dense(units=3, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(lr=0.02)
    ann.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

    ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

    weights = ann.get_weights()
    return ann

# Predicting Test Set Results

In [52]:
def prediction(ann, x_test):
    y_pred = ann.predict(x_test)
    # boolean result of whether predicted probability is larger than 0.5
    # result > 0.5 = 1, result <= 0.5 = 0
    y_pred_binary = (y_pred > 0.5)
    return y_pred, y_pred_binary

# Obtaining Confusion Matrix and Accuracy Score for predictions

In [53]:
def analysis(y_test, y_pred_binary):
    # To evaluate accuracy, a vector of labels is needed
    # np.argmax(x, axis=1) - to output correct labels
    from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

    # Confusion Matrix: Columns = 0, 1 (Predicted Label)
    # Confusion Matrix: Rows = 0, 1 (Actual Label)
    cm = multilabel_confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1))
    print(cm)

    print()
    print("Accuracy Score: ")
    print(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1)))
    print("----------------------------")

# Method execution

In [54]:
df = dfs[4]
print("Stock: ")
print(df.name)

x, y = split_dataset(df)
y = encode_var(y)
x_train, x_test, y_train, y_test = train_test_sp(x, y)
ann = machine_learning(x_train, y_train)
y_pred, y_pred_binary = prediction(ann, x_test)
analysis(y_test, y_pred_binary)



Stock: 
AWK.csv
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
Train on 402 samples
Epoch 1/100
402/402 [==============================] - 1s 3ms/sample - loss: 0.7169 - accuracy: 0.6468
Epoch 2/100
402/402 [==============================] - 0s 69us/sample - loss: 0.6488 - accuracy: 0.6667
Epoch 3/100
402/402 [==============================] - 0s 82us/sample - loss: 0.6291 - accuracy: 0.6667
Epoch 4/100
402/402 [==============================] - 0s 97us/sample - loss: 0.6173 - accuracy: 0.6667
Epoch 5/100
402/402 [==============================] - 0s 84us/sample - loss: 0.6104 - accuracy: 0.6667